In [35]:
import requests
from datetime import datetime, timezone, timedelta
import math

In [36]:
# Starting URL for MBTA  API Call.
url = "https://api-v3.mbta.com"

In [37]:
# Make a 'Get' request for the routes.
# Center on Watertown Square for testing. sort by soonest departure time. include stop information
center_loc = [42.365165,-71.1850722]
query = "/predictions?sort=departure_time&include=stop%2Croute&filter%5Blatitude%5D=42.365165&filter%5Blongitude%5D=-71.1850722&filter%5Bradius%5D=0.005"
queryResults = requests.get(url+query)

In [38]:
# Get the JSON of the 'Get' request.
predictions = [x for x in queryResults.json()['data'] if x['attributes']['departure_time'] != None]
stops = {x['id']:x['attributes'] for x in queryResults.json()['included'] if x['type']=="stop"}
routes = {x['id']:x['attributes'] for x in queryResults.json()['included'] if x['type']=="route"}

In [39]:
#Get the distance from the location. Using watertown square for testing
for stop in stops:
  # print(stops[stop]['name'])
  # print(stops[stop]['latitude'])
  # print(stops[stop]['longitude'])
  stop_loc = [stops[stop]['latitude'],stops[stop]['longitude']]
  # print(math.dist(stop_loc, center_loc))
  stops[stop]['distance'] = math.dist(stop_loc, center_loc)

In [40]:
# each trip gets one prediction in the newPredictionDict - the one closest to the location
newPredictionDict = {}
for prediction in predictions:
  trip = str(prediction['relationships']['trip']['data']['id'])
  if trip in newPredictionDict:
    # if the trip has an entry in the list, compare the distance between the stops
    oldStopID = newPredictionDict[trip]['relationships']['stop']['data']['id']
    newStopID = prediction['relationships']['stop']['data']['id']
    # if stop in the dict is farther than the new stop, replace it
    if (stops[oldStopID]['distance'] > stops[newStopID]['distance']):
      newPredictionDict[trip] = prediction
  else:
    # otherwise, add the trip to the dict
    newPredictionDict[trip] = prediction

In [41]:
# time display configurations
timezone_offset = -4.0
tzinfo = timezone(timedelta(hours=timezone_offset))
now = datetime.now(tzinfo)
# datetime.fromisoformat(predictions[1]['attributes']['departure_time']) - now

# Build a new dict. Each Route gets two predictions for each direction
displayDict = {}
for predict in newPredictionDict:
  x = newPredictionDict[predict]
  d = datetime.fromisoformat(x['attributes']['departure_time'])
  # only display trips in the future in the next 20 minutes
  if d < now + timedelta(minutes=20):
    route_ID = x['relationships']['route']['data']['id']
    route = routes[route_ID]
    stop_ID = x['relationships']['stop']['data']['id']
    stop = stops[stop_ID]
    routeDisplayKey = route['short_name'] + "_" + str(x['attributes']['direction_id'])
    if not(routeDisplayKey in displayDict):
      routeDisplay = {
          "bus":route['fare_class'] + " " + route['short_name'] + " " + route['direction_names'][x['attributes']['direction_id']],
          "destination":route['direction_destinations'][x['attributes']['direction_id']],
          "stop":stop['name'],
          "departure_time":datetime.fromisoformat(x['attributes']['departure_time']).strftime("%I:%M %p")
      }
      displayDict[routeDisplayKey] = [routeDisplay]
    elif (len(displayDict[routeDisplayKey])<2):
      routeDisplay = {
          "bus":route['fare_class'] + " " + route['short_name'] + " " + route['direction_names'][x['attributes']['direction_id']],
          "destination":route['direction_destinations'][x['attributes']['direction_id']],
          "stop":stop['name'],
          "departure_time":datetime.fromisoformat(x['attributes']['departure_time']).strftime("%I:%M %p")
      }
      displayDict[routeDisplayKey].append(routeDisplay)
    # print(route['fare_class'] + " " + route['short_name'] + " " + route['direction_names'][x['attributes']['direction_id']] + " towards " + route['direction_destinations'][x['attributes']['direction_id']])
    # depart_time = datetime.fromisoformat(x['attributes']['departure_time'])
    # print("Departs from " + stop['name']+ " at " + depart_time.strftime("%I:%M %p"))
    # print(" - " )


In [42]:
for route in displayDict:
  line = displayDict[route]
  print(line[0]["bus"] + " toward " + line[0]["destination"])
  for stop in line:
    print("Departs from " + stop['stop']+ " at " + stop['departure_time'])
  print("--")

Local Bus 57 Inbound toward Kenmore Station
Departs from Watertown Yard at 09:50 PM
Departs from Watertown Yard at 10:00 PM
--
Local Bus 71 Outbound toward Watertown Square
Departs from Mt Auburn St @ Main St at 09:52 PM
--
Local Bus 70 Inbound toward University Park
Departs from Main St @ Cross St at 09:55 PM
--
Local Bus 71 Inbound toward Harvard Station
Departs from Watertown Sq Terminal at 09:58 PM
--
